In [14]:
import os
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.transforms.functional as TF

def extract_labels(labels):
    # Map emotion labels to integers
    label_mapping = {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}
    return np.array([label_mapping[label] for label in labels])

class EmotionDataset(Dataset):
    def __init__(self, dataset_path, transform=None):
        self.data, self.labels = self.load_data(dataset_path)
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label
    
    @staticmethod
    def load_data(dataset_path):
        data = []
        labels = []

        for emotion_folder in os.listdir(dataset_path):
            emotion_path = os.path.join(dataset_path, emotion_folder)
            for img_name in os.listdir(emotion_path):
                img_path = os.path.join(emotion_path, img_name)
                img = cv2.imread(img_path)
                data.append(img)
                labels.append(emotion_folder)

        return np.array(data), extract_labels(labels)
    
    


In [15]:
if torch.backends.mps.is_available():
	device=torch.device("mps")
elif torch.cuda.is_available():
	device=torch.device("cuda")
else:
	device=torch.device("cpu")

print(device)

cuda


In [16]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [17]:
train_path = 'FER2013/train'
absolute_train_path = os.path.abspath(train_path)

test_path = 'FER2013/test'
absolute_test_path = os.path.abspath(test_path)

train_dataset = EmotionDataset(absolute_train_path, transform=transform)
test_dataset = EmotionDataset(absolute_test_path, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [18]:
import torch.nn as nn

# Define a custom CNN model
class EmotionCNN(nn.Module):
    def __init__(self, num_classes=7):
        super(EmotionCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(128 * 56 * 56, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(512, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x

# Instantiate the CNN model
model = EmotionCNN(num_classes=7).to(device)

In [19]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [20]:
import torch.optim as optim

# Assuming you have a GPU available, check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move your model to the GPU
model.to(device)

# Define your loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

# Training loop
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        # Move inputs and labels to the GPU
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        
        # Convert labels to torch.long
        labels = labels.long()
        
        # Convert outputs to torch.long if necessary
        # outputs = outputs.long()  # Uncomment this line if needed
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')


Epoch 1/10, Loss: 1.8723430633544922
Epoch 2/10, Loss: 2.1659233570098877
Epoch 3/10, Loss: 1.767133355140686
Epoch 4/10, Loss: 1.6139131784439087
Epoch 5/10, Loss: 1.7873523235321045
Epoch 6/10, Loss: 1.695610761642456
Epoch 7/10, Loss: 1.898927092552185
Epoch 8/10, Loss: 1.8719546794891357
Epoch 9/10, Loss: 1.6050500869750977
Epoch 10/10, Loss: 2.3958637714385986


In [21]:
for inputs, labels in train_loader:
    print(type(inputs))
    print(type(labels))
    print(inputs)
    print(labels)
    break

<class 'torch.Tensor'>
<class 'torch.Tensor'>
tensor([[[[-1.5870, -1.5870, -1.5870,  ..., -1.7412, -1.7412, -1.7412],
          [-1.5870, -1.5870, -1.5870,  ..., -1.7412, -1.7412, -1.7412],
          [-1.5870, -1.5870, -1.5870,  ..., -1.7412, -1.7412, -1.7412],
          ...,
          [-1.6042, -1.6042, -1.5870,  ..., -1.6727, -1.6727, -1.6727],
          [-1.6042, -1.6042, -1.5870,  ..., -1.6727, -1.6727, -1.6727],
          [-1.6042, -1.6042, -1.5870,  ..., -1.6727, -1.6727, -1.6727]],

         [[-1.4930, -1.4930, -1.4930,  ..., -1.6506, -1.6506, -1.6506],
          [-1.4930, -1.4930, -1.4930,  ..., -1.6506, -1.6506, -1.6506],
          [-1.4930, -1.4930, -1.4930,  ..., -1.6506, -1.6506, -1.6506],
          ...,
          [-1.5105, -1.5105, -1.4930,  ..., -1.5805, -1.5805, -1.5805],
          [-1.5105, -1.5105, -1.4930,  ..., -1.5805, -1.5805, -1.5805],
          [-1.5105, -1.5105, -1.4930,  ..., -1.5805, -1.5805, -1.5805]],

         [[-1.2641, -1.2641, -1.2641,  ..., -1.4210, -1.

In [22]:
model.eval()
test_accuracy = 0  
num_correct = 0
num_samples = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.long().to(device)
        
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        num_correct += (preds == labels).sum()
        num_samples += labels.size(0)
        acc = float(num_correct) / num_samples
        test_accuracy += acc

print(f'Test Accuracy: {test_accuracy / len(test_loader):.4f}')

Test Accuracy: 0.2411
